데이터 전처리

In [21]:
import pandas as pd

# Load dataset
file_path = 'kma_2022_seoul_noNULL_noDuplicated.csv'  # 파일 경로를 지정하세요.
data = pd.read_csv(file_path)

# Convert 'TM' column to datetime format
data['TM'] = pd.to_datetime(data['TM'], format='%Y%m%d%H%M')

# Select relevant columns for prediction
selected_columns = ['STN','TM', 'WD', 'WS', 'PA', 'PS', 'TA', 'TD', 'HM', 'CA_TOT', 'CA_MID', 'VS', 'TS', 'TE_005', 'TE_01', 'TE_02','TE_03', 'PV', 'PM10']
processed_data = data[selected_columns]

# Sort by time
processed_data = processed_data.sort_values('TM').reset_index(drop=True)

BASE MODEL : ARIMA 모델

In [23]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

arima_data = processed_data[['TM', 'PM10']].set_index('TM')

train_size = int(len(arima_data) * 0.7)
val_size = int(len(arima_data) * 0.15)

train_arima = arima_data.iloc[:train_size]['PM10']
y_val = arima_data.iloc[train_size:train_size + val_size]['PM10']
y_test = arima_data.iloc[train_size + val_size:]['PM10']
arima_order = (5, 1, 0)  # (p, d, q) 값 조정 가능
arima_model = ARIMA(train_arima, order=arima_order)
arima_fit = arima_model.fit()

# 예측
forecast_val = arima_fit.forecast(steps=len(y_val))
forecast_test = arima_fit.forecast(steps=len(y_test))

# 평가
val_rmse = np.sqrt(mean_squared_error(y_val, forecast_val))
val_mae = mean_absolute_error(y_val, forecast_val)
test_rmse = np.sqrt(mean_squared_error(y_test, forecast_test))
test_mae = mean_absolute_error(y_test, forecast_test)

print(f"ARIMA Validation RMSE: {val_rmse}, MAE: {val_mae}")
print(f"ARIMA Test RMSE: {test_rmse}, MAE: {test_mae}")


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


ARIMA Validation RMSE: 17.846696096140953, MAE: 13.52487821179158
ARIMA Test RMSE: 45.46175865948999, MAE: 22.593982505619035


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


MLP model

In [25]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Features (X) and target (y)
X = processed_data.drop(columns=['TM', 'PM10'])
y = processed_data['PM10']

# Split into training, validation, and test sets
train_size = 0.7
X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size=train_size, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False)


# 데이터 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# MLP 모델 설계
mlp_model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer
])

mlp_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# 학습
history = mlp_model.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=50,
    batch_size=32,
    verbose=1
)

# 평가
train_loss = history.history['loss'][-1]  # 마지막 epoch의 train loss
val_loss, val_mae = mlp_model.evaluate(X_val_scaled, y_val, verbose=0)
test_loss, test_mae = mlp_model.evaluate(X_test_scaled, y_test, verbose=0)

print(f"MLP Train Loss (RMSE): {np.sqrt(train_loss)}")
print(f"MLP Validation Loss (RMSE): {np.sqrt(val_loss)}, MAE: {val_mae}")
print(f"MLP Test Loss (RMSE): {np.sqrt(test_loss)}, MAE: {test_mae}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 1443.2198 - mae: 26.9667 - val_loss: 211.3288 - val_mae: 11.4414
Epoch 2/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 428.2059 - mae: 14.6291 - val_loss: 180.6876 - val_mae: 10.4324
Epoch 3/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 387.4608 - mae: 13.6831 - val_loss: 175.6092 - val_mae: 10.2711
Epoch 4/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 381.9409 - mae: 13.0660 - val_loss: 180.7500 - val_mae: 10.3706
Epoch 5/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 367.0401 - mae: 12.8042 - val_loss: 179.6760 - val_mae: 10.2957
Epoch 6/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 308.4959 - mae: 12.3601 - val_loss: 177.1327 - val_mae: 10.3087
Epoch 7/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 337.6331 - mae: 12.3774 - val_loss: 186.2342 - val_mae: 10.4967
Epoch 8/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 292.5948 - mae: 11.9192 - val_loss: 188.9876 - val_mae: 10.5276
Epoch 9

과적합

In [26]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Features (X) and target (y)
X = processed_data.drop(columns=['TM', 'PM10'])
y = processed_data['PM10']

# Split into training, validation, and test sets
train_size = 0.7
X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size=train_size, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False)


# 데이터 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# MLP 모델 설계
mlp_model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer
])

mlp_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# 학습
history = mlp_model.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=200,
    batch_size=32,
    verbose=1
)

# 평가
train_loss = history.history['loss'][-1]  # 마지막 epoch의 train loss
val_loss, val_mae = mlp_model.evaluate(X_val_scaled, y_val, verbose=0)
test_loss, test_mae = mlp_model.evaluate(X_test_scaled, y_test, verbose=0)

print(f"MLP Train Loss (RMSE): {np.sqrt(train_loss)}")
print(f"MLP Validation Loss (RMSE): {np.sqrt(val_loss)}, MAE: {val_mae}")
print(f"MLP Test Loss (RMSE): {np.sqrt(test_loss)}, MAE: {test_mae}")

Epoch 1/200


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


187/187 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 1127.1495 - mae: 23.9714 - val_loss: 202.2554 - val_mae: 11.2551
Epoch 2/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 441.2302 - mae: 14.4473 - val_loss: 170.3916 - val_mae: 10.1052
Epoch 3/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 375.2069 - mae: 13.2798 - val_loss: 176.8492 - val_mae: 10.2579
Epoch 4/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 371.8944 - mae: 13.1660 - val_loss: 167.6781 - val_mae: 9.9522
Epoch 5/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 364.3821 - mae: 12.9788 - val_loss: 167.5808 - val_mae: 9.9430
Epoch 6/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 322.7094 - mae: 12.5472 - val_loss: 188.7257 - val_mae: 10.5631
Epoch 7/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 294.0996 - mae: 12.0017 - val_loss: 187.3437 - val_mae: 10.4457
Epoch 8/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 334.3861 - mae: 12.2955 - val_loss: 179.5086 - val_mae: 10.4195
Epoch 9/200
